<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - KNN Item Item MRR</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [3]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, item_knn as knn
from lenskit import topn
import pandas as pd
import numpy as np

ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [12]:
def model_trainer(neighbours):
    neighbours=int(neighbours)
    print(neighbours)
    all_recs = []
    test_data = []
    algo_ii = knn.ItemItem(neighbours,aggregate='weighted-average')
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test)
        all_recs.append(eval('ItemItem', algo_ii, train, test))
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla = topn.RecListAnalysis()
    rla.add_metric(topn.recip_rank)
    results = rla.compute(all_recs, test_data)
    return -results["recip_rank"].mean()

In [13]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(x):
    return {'loss': model_trainer(x), 'status': STATUS_OK }

average=[[] for _ in range(10)]

for y in range(len(average)):
    best = fmin(objective,
        space=hp.uniform('x', 1, 50),
        algo=tpe.suggest,
        max_evals=10
               )
    average[y].append(best)

8                                                                                                                      
40                                                                                                                     
49                                                                                                                     
49                                                                                                                     
46                                                                                                                     
47                                                                                                                     
2                                                                                                                      
33                                                                                                                     
18                                      

8                                                                                                                      
40                                                                                                                     
35                                                                                                                     
10                                                                                                                     
17                                                                                                                     
31                                                                                                                     
39                                                                                                                     
2                                                                                                                      
100%|███████████████████████████████████

In [21]:
average = [[{'x': 8.934221300944941}],
 [{'x': 9.37255731841272}],
 [{'x': 13.960251652364823}],
 [{'x': 12.676041196908512}],
 [{'x': 7.221128168169952}],
 [{'x': 15.574599693512226}],
 [{'x': 10.013577817451978}],
 [{'x': 10.327122032772271}],
 [{'x': 11.863009967836662}],
 [{'x': 21.257435795649442}]]

In [22]:
flat_list = [item for sublist in average for item in sublist]

In [23]:
mrr = [0.12289842438871736,0.1196460987909784,0.12322030997883539,0.12071170013751349,0.11792344668643438,0.11326044370964773,
       0.11691948491900184,0.11929380395051468,0.11919123582415096,0.10551181571507753]
mrr = pd.DataFrame(mrr)

In [24]:
average = pd.DataFrame.from_dict(flat_list)
metric = pd.concat([average,mrr],axis=1)
metric.columns = ['Neighbours','MRR']
metric

,Neighbours,MRR
0,8.934221,0.122898
1,9.372557,0.119646
2,13.960252,0.123220
3,12.676041,0.120712
4,7.221128,0.117923
5,15.574600,0.113260
6,10.013578,0.116919
7,10.327122,0.119294
8,11.863010,0.119191
9,21.257436,0.105512


In [27]:
metric.mean()

Neighbours    12.119994
MRR            0.117858
dtype: float64